In [31]:
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import pandas as pd
import numpy as np

In [2]:
with open('dataset_emotion_preprocessed.pickle', 'rb') as f:
   dataset_emotion = pickle.load(f)

In [4]:
dataset_emotion.columns

Index(['tokenized_tweets', 'emotion', 'processed'], dtype='object')

In [16]:
tokenizer = Tokenizer(num_words=1500, split=' ')

tokenizer.fit_on_texts(dataset_emotion['processed'].values)

X = tokenizer.texts_to_sequences(dataset_emotion['processed'])

X = pad_sequences(X)

In [40]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2000, 300,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           600000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 30, 300)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1182      
Total params: 990,830
Trainable params: 990,830
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(47813, 30) (47813, 6)
(11954, 30) (11954, 6)


In [42]:
Y = pd.get_dummies(dataset_emotion['emotion']).values

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [45]:
batch_size = 32
model.fit(X_train, y_train, epochs = 100, batch_size=batch_size, verbose = 2)

Epoch 1/100
